In [2]:
# -*- coding: utf-8 -*-
import sys
import os
import unicodedata
import string
import urllib2

import numpy as np
import pandas as pd

# Agrego al path la carpeta modulos
sys.path.insert(0, os.path.abspath("../../modulos"))
from LexAnalyser import LexAnalyser

In [3]:
url = 'http://www.gutenberg.org/cache/epub/2000/pg2000.txt'
response = urllib2.urlopen(url)
raw = response.read()
start = raw.find('El ingenioso hidalgo don Quijote de la Mancha')
end = raw.rfind('End of Project Gutenberg')
raw = raw[start:end]
print len(raw)

2178990


In [15]:
vocabulario = {}
la = LexAnalyser({})
terms = la.analyse(raw.decode("UTF-8"))["terms"]
for t in terms:
    if t not in vocabulario:
        vocabulario[t] = 1
    else:
        vocabulario[t] += 1
print "Vocabulario (extracto)", [{key: val} for key, val in vocabulario.items()[:10]]
print "Cantidad de terminos: ", len(terms)

Vocabulario (extract) [{u'consuelome': 1}, {u'canes': 3}, {u'igual': 16}, {u'indignacion': 2}, {u'hermana': 11}, {u'hermano': 106}, {u'gastara': 1}, {u'acababamos': 1}, {u'dictado': 1}, {u'compuesta': 11}]
Cantidad de terminos:  381215


In [7]:
df = pd.DataFrame(index = vocabulario.keys(),
                  columns = ['Frecuencia','Ranking','C=F*R'],
                  dtype = 'float64')

for a in df.index:
    df.loc[a,'Frecuencia'] = vocabulario[a]

df['Ranking'] = df['Frecuencia'].rank(method = 'min', ascending = False)

df['C=F*R'] = df['Frecuencia'] * df['Ranking']

df.sort_values(['Frecuencia'], ascending=False).head()

,Frecuencia,Ranking,C=F*R
que,21475.0,1.0,21475.0
de,18298.0,2.0,36596.0
y,18188.0,3.0,54564.0
la,10363.0,4.0,41452.0
a,9823.0,5.0,49115.0


In [8]:
# Agrego alguna data extra 
n = df['Frecuencia'].sum()
df['Frec. Acum.'] = df['Frecuencia'].sort_values(ascending=False).cumsum()
df['Frec. Rel.'] = df['Frecuencia'] / n
df['Frec. Rel. Acum.'] = df['Frec. Acum.'] / n
df.sort_values(['Frecuencia'], ascending=False).head()

,Frecuencia,Ranking,C=F*R,Frec. Acum.,Frec. Rel.,Frec. Rel. Acum.
que,21475.0,1.0,21475.0,21475.0,0.056333,0.056333
de,18298.0,2.0,36596.0,39773.0,0.047999,0.104332
y,18188.0,3.0,54564.0,57961.0,0.047711,0.152043
la,10363.0,4.0,41452.0,68324.0,0.027184,0.179227
a,9823.0,5.0,49115.0,78147.0,0.025768,0.204995


In [9]:
# Selecciono solo aquella data que tiene frecuencia < 5
df_menor_5 = df[(df['Frecuencia'] < 5.0)]
df_menor_5.sort_values(['Frec. Acum.'], ascending=True).to_csv('extras/quijote_frecuencia_menor_5.csv')
df_menor_5.sort_values(['Frec. Acum.'], ascending=True).head()

,Frecuencia,Ranking,C=F*R,Frec. Acum.,Frec. Rel.,Frec. Rel. Acum.
guardese,4.0,5270.0,21080.0,353749.0,0.00001,0.927951
vituperio,4.0,5270.0,21080.0,353753.0,0.00001,0.927962
viedma,4.0,5270.0,21080.0,353757.0,0.00001,0.927972
proezas,4.0,5270.0,21080.0,353761.0,0.00001,0.927983
endereza,4.0,5270.0,21080.0,353765.0,0.00001,0.927993


In [10]:
porcentaje_omitido = (1 - df_menor_5["Frec. Rel. Acum."].min()) * 100
print "Quitando del índice aquellos terminos cuya frecuencia es menor que 5 se omite el %.2f por ciento del total de términos" % porcentaje_omitido

Quitando del índice aquellos terminos cuya frecuencia es menor que 5 se omite el 7.20 por ciento del total de términos


In [43]:
# PROPORCION SEGUN ZIPF
constante = df['C=F*R'].mean()
#constante = df[df['Frecuencia'] == frecuencia]["C=F*R"].mean()
frecuencia = 4.0
beta = -np.polyfit(np.log(df['Ranking']), np.log(df['Frecuencia']), deg = 1)[0]

# Se resuelve siguiente ecuacion:
# ranking = (frecuencia * constante) ^ 1/beta
f_por_c = constante * frecuencia
raiz_beta = 1 / beta
ranking = np.power(f_por_c, raiz_beta)

df_zipf = df[df['Ranking'] >= ranking]
df_zipf.sort_values(['Frec. Acum.'], ascending=True).head()

# OTRA FORMULA: Ranking(frecuancia) = beta * cantidad_terminos_repetidos / frecuencia
#r = beta * (len(terms) / frecuencia)
#print r
#df_zipf = df[df['Ranking'] >= r]
#df_zipf.sort_values(['Frec. Acum.'], ascending=True).head()

,Frecuencia,Ranking,C=F*R,Frec. Acum.,Frec. Rel.,Frec. Rel. Acum.
tras,83.0,460.0,38180.0,275479.0,0.000218,0.722634
nuevas,83.0,460.0,38180.0,275562.0,0.000218,0.722852
pueden,83.0,460.0,38180.0,275645.0,0.000218,0.723070
ti,82.0,463.0,37966.0,275727.0,0.000215,0.723285
sazon,82.0,463.0,37966.0,275809.0,0.000215,0.723500


In [19]:
porcentaje_omitido = (1 - df_zipf["Frec. Rel. Acum."].min()) * 100
print "Segun Zipf se omite el %.2f por ciento del total de términos" % porcentaje_omitido

Segun Zipf se omite el 11.12 por ciento del total de términos


In [20]:
# PROPORCION SEGUN ZIPF
constante = df['C=F*R'].mean()
ranking = 5270
beta = -np.polyfit(np.log(df['Ranking']), np.log(df['Frecuencia']), deg = 1)[0]

# Se resuelve siguiente ecuacion:
# frecuencia =  constante / (ranking ^ beta) 
frecuencia = constante / np.power(ranking, beta)
print frecuencia

0.10916656585
